In [1]:
import pandas as pd
import numpy as np
import math

My own classifer

In [2]:
class MyGaussianBayes:
    def __init__(self): 
        self.trainX = None
        self.trainY = None
        self.feature_cols = []
        self.class_labels = []
        # the Gaussian for calculating p(value|some_y_label)
        # using dict{ylabel: pandas series}. The pandas series is {feature_col_name: p}
        self.mean_dict = {}
        self.stds_dict = {}
        # probability of each y label (prior)
        self.prob_label = {}
        
    
    '''
    fit is a training process
    '''
    def fit(self, trainX, trainY):
        """
        :param trainX: Pandas DataFrame
        :param trainY: Pandas DataFrame
        :return: 
        """
        self.trainX = trainX
        self.trainY = trainY
        self.feature_cols = trainX.columns.tolist()
        
        y_unique_labels = self.trainY.unique()
        total_num_rows = len(self.trainY)
        
        for possible_y in y_unique_labels:
            self.class_labels.append(possible_y)
            #y=0时的所有行构成的矩阵，第二遍取出y=1时所有行构成的矩阵
            df_with_possible_y = self.trainX.loc[self.trainY == possible_y, :] 
            self.prob_label[possible_y] = len(df_with_possible_y) / total_num_rows
            
            means = df_with_possible_y.mean()
            self.mean_dict[possible_y] = means
            stds = df_with_possible_y.std()
            self.stds_dict[possible_y] = stds
    
    '''calculate normal disribution'''
    def normpdf(self, x, mean, sd):
        var = float(sd)**2
        denom = (2*math.pi*var)**.5
        num = math.exp(-(float(x)-float(mean))**2/(2*var))
        return num/denom
            
    def predict(self, testX):
        predict_y = []
        
        for i in range(testX.shape[0]):
            row = testX.iloc[i, :] #row is a series,like a dict
            max_prob = 0
            true_y = 0
            for possible_y in self.class_labels:
                prob_product = 1
                prob_c = self.prob_label[possible_y]
                for col_name in self.feature_cols:
                    m = self.mean_dict[possible_y][col_name]
                    s = self.stds_dict[possible_y][col_name]
                    current_value = row[col_name]
                    # if I need to call a class method in this class method, should apply self.method_name
                    prob_feature = self.normpdf(current_value, m, s)
                    prob_product *= prob_feature
                prob_product *= prob_c
                if prob_product > max_prob:
                    max_prob = prob_product
                    true_y = possible_y
            predict_y.append(true_y)
        return predict_y
            
                

train_data = pd.read_csv("hw1_trainingset.csv")
test_data = pd.read_csv("hw1_testset.csv")
my_nb = MyGaussianBayes()
my_nb.fit(
    train_data.iloc[:, : -1],
    train_data.iloc[:, -1]
)

predicted_y = my_nb.predict(test_data)

test_data['Label'] = predicted_y
test_data.to_csv("my_gaussian_bayes_predict_result.csv")

My own crossvalidation

In [3]:
from cross_validation import my_cross_validation
scores = my_cross_validation(MyGaussianBayes, train_data.iloc[:, : -1], train_data.iloc[:, -1], 10)
print("The cv scores from my cross validation is " + str(scores) + " and the average is " 
      + str(sum(scores) / len(scores)) )

The cv scores from my cross validation is [0.3483483483483483, 0.3493975903614458, 0.32, 0.2874251497005988, 0.3861671469740634, 0.31347962382445144, 0.3323442136498516, 0.31736526946107785, 0.2546583850931677, 0.3177570093457944] and the average is 0.3226942736758799


Using sklean

In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
train_data = pd.read_csv("hw1_trainingset.csv")
test_data = pd.read_csv("hw1_testset.csv")

train_data = train_data.sample(frac=1, random_state=42)
trainX = train_data.iloc[:, : -1]
trainY = train_data.iloc[:, -1]

clf = GaussianNB()
clf.fit(trainX, trainY)

testX = test_data
predicted_y = clf.predict(testX)
test_data['Label'] = predicted_y
test_data.to_csv("sklearn_gaussian_bayes_predict_result.csv")

scores = cross_val_score(GaussianNB(), trainX, trainY,
                scoring="f1",
                cv=10)

print("The cv scores from sklearn is " + str(scores) + " and the average is " 
      + str(sum(scores) / len(scores)) )

The cv scores from sklearn is [0.33038348 0.28025478 0.34131737 0.3313253  0.31858407 0.28037383
 0.3081571  0.34636872 0.29878049 0.32615385] and the average is 0.31616989756828723
